In [2]:
from matplotlib import cm
from skimage import measure, feature, filters, transform, morphology,segmentation
import numpy as np
import pandas as pd
import read_roi as rr
import matplotlib.pyplot as plt
import glob,os
import pickle as pk
import tifffile
from colony_tracker import *

In [3]:
pks = sorted(glob.glob('/Users/jz-rolling/Desktop/Size_calculation/20220118/Cropped_20220120/*.pk'))
record_f = '/Users/jz-rolling/Desktop/Size_calculation/20220118/Colonies_20220121/Tracking_records/'
colony_f = '/Users/jz-rolling/Desktop/Size_calculation/20220118/Colonies_20220121/Colonies/'
ref = pd.read_excel('/Users/jz-rolling/Desktop/Size_calculation/20220118/Plate_ID_8_Drugs.xlsx')

In [ ]:
# filter colony patches
# export colony profiles
for p in pks:
    header = p.split('/')[-1].split('.')[0]
    plate,col,row = header.split('_')
    plate = int(plate[-1])
    col = int(col[-1])
    row = int(row[-1])
    ref_info = list(ref[(ref['Plate']=='Plate {}'.format(plate))&\
                        (ref['Column']==col)].values[0])
    cond,strain = ref_info[1],ref_info[2]
    dst_f = '{}{}_{}_{}/'.format(colony_f,cond,strain,header)
    f_withscalebar = '{}withscalebar/'.format(dst_f)
    f_nobar = '{}noscalebar/'.format(dst_f)
    f_colored = '{}labeled/'.format(dst_f)
    #f_raw = '{}nolabel/'.format(dst_f)
    if not os.path.isdir(dst_f):
        os.mkdir(dst_f)
    for f in [f_withscalebar,f_nobar,f_colored]:
        if not os.path.isdir(f):
            os.mkdir(f)
    d = pk.load(open(p,'rb'))
    c = Colony(d['Images'],d['Masks'],d['Edges'],d['Days'])
    c.fill_holes(area_threshold=50)
    c.render_highcontrast_images(p1=70)
    c.link_colonies()
    c.measure_growth()
    c.filter_colonies()
    c.label_colonies()
    c.inspect_colonies()
    
    for i,img in enumerate(c.high_contrast_images):
        h,w = img.shape
        d = c.dates[i]
        fig=plt.figure(figsize=(3,3))
        ax=fig.add_subplot()
        ax.imshow(img,vmin=0,vmax=255,cmap='gist_gray')
        plt.axis('off')
        plt.savefig(f_nobar+'{}.png'.format(d),bbox_inches='tight',dpi=160)
        plt.plot([0.95*w-50,0.95*w],[0.1*h,0.1*h],color='w',lw=5)
        plt.savefig(f_withscalebar+'{}.png'.format(d),bbox_inches='tight',dpi=160)
        plt.close()
        fig2 = plt.figure(figsize=(3,3))
        ax2 = fig2.add_subplot()
        plt.axis('off')
        ax2.imshow(c.colored_labels[i])
        ax2.text(0.05,0.95,
                'Day {}'.format(d),
                ha='left',
                va='top',
                transform = ax2.transAxes,
                fontweight='bold',
                fontname='Arial',
                fontsize=14,color='w')
        plt.savefig(f_colored+'{}.png'.format(d),bbox_inches='tight',dpi=160)
        plt.close()        
    contact_info = []
    growths = []
    colony_ids = []
    days = []
    for k,g in c.colony_growths.items():
        touched = c.touched[k]
        colony_ids += ['{}_{}'.format(header,k)]*len(g)
        contact_info += list(touched)
        growths += list(g)
        days += list(c.dates)
    colony_df = pd.DataFrame({'Colony_id':colony_ids,
                              'Area':growths,
                              'Fraction_contact':contact_info,
                              'Days':days})

    colony_df[ref.columns] = [ref_info]*len(colony_df)
    colony_df['Row'] = row
    colony_df.to_excel(dst_f+'colony_profiles.xlsx')